In [1]:
import pandas as pd


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gbjs0748\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#read dataset
df = pd.read_csv('intent.csv',encoding= 'latin1',sep=';')

In [5]:
df.head()

,phrases,intention,class
0,activer un renvoi d'appel,renvoi d'appel,1
1,demande à djingo pro de renvoyer les appels,renvoi d'appel,1
2,de renvoyer les appels,renvoi d'appel,1
3,aide moi à renvoyer mes appels,renvoi d'appel,1
4,je veux renvoyer mes appels,renvoi d'appel,1


In [6]:
#new df with only 2 columns
df.columns

Index(['phrases', 'intention', 'class'], dtype='object')

In [7]:
df['phrases']

0                       activer un renvoi d'appel
1     demande à djingo pro de renvoyer les appels
2                          de renvoyer les appels
3                  aide moi à renvoyer mes appels
4                     je veux renvoyer mes appels
5                               Renvoi mes appels
6                      Renvoi mes appels vers le 
7       je veux que tu renvoi mes appels vers le 
8      je veux que tu renvoi mes appels vers mon 
9                     renvoi mes appels vers mon 
10            je veux annuler les renvoi d appels
11             je veux annuler le renvoi d appels
12               annule les renvois de mes appels
13                  annule les renvois des appels
14                    annule les renvois d appels
15                 Annule le renvoi de mes appels
16         je souhaite annuler le renvoi d'appels
17                         annule renvoi d'appels
18                     annule mon renvoi d'appels
19                      annule le renvoi d'appels


In [8]:
df_intents = df[['phrases','class']]

In [9]:
df_intents.head()

,phrases,class
0,activer un renvoi d'appel,1
1,demande à djingo pro de renvoyer les appels,1
2,de renvoyer les appels,1
3,aide moi à renvoyer mes appels,1
4,je veux renvoyer mes appels,1


In [10]:
df_intents.tail()

,phrases,class
23,je suis rentré,3
24,je suis de retour,3
25,de retour au bureau,3
26,je suis là,3
27,je suis déjà là,3


In [11]:
#tokenisation
phrases = df.phrases
#remove all things but characters and space
phrases = phrases.str.replace('[^A-z]',' ').str.replace(' +', ' ').str.strip()
print(phrases)

0                     activer un renvoi d appel
1     demande djingo pro de renvoyer les appels
2                        de renvoyer les appels
3                  aide moi renvoyer mes appels
4                   je veux renvoyer mes appels
5                             Renvoi mes appels
6                     Renvoi mes appels vers le
7      je veux que tu renvoi mes appels vers le
8     je veux que tu renvoi mes appels vers mon
9                    renvoi mes appels vers mon
10          je veux annuler les renvoi d appels
11           je veux annuler le renvoi d appels
12             annule les renvois de mes appels
13                annule les renvois des appels
14                  annule les renvois d appels
15               Annule le renvoi de mes appels
16       je souhaite annuler le renvoi d appels
17                       annule renvoi d appels
18                   annule mon renvoi d appels
19                    annule le renvoi d appels
20          je veux annuler mon renvoi d

In [12]:
#remove stop words
stopW = stopwords.words('french')
#print(stopW)

In [13]:
#exclude stopswords
df_intents['phrases_sans_stop_words'] = df_intents['phrases'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopW)]))
#df = df.drop('phrases_sans_stop_wors') 


In [56]:
#df_intents = df_intents.drop('phrases_sans_stop_wors', axis = 1)

In [14]:
df_intents.head()

,phrases,class,phrases_sans_stop_words
0,activer un renvoi d'appel,1,activer renvoi d'appel
1,demande à djingo pro de renvoyer les appels,1,demande djingo pro renvoyer appels
2,de renvoyer les appels,1,renvoyer appels
3,aide moi à renvoyer mes appels,1,aide renvoyer appels
4,je veux renvoyer mes appels,1,veux renvoyer appels


In [15]:
#sinon on fait un stemmer : stemmer permet de ne garder que la racine d'un mot
def steammer_phrase(phrases):
    
    stemmer = FrenchStemmer()
    tokens = word_tokenize(phrases)
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

In [16]:
df_intents['stemmed_phrases'] = df_intents['phrases_sans_stop_words'].apply(lambda phrases: steammer_phrase(phrases))

In [17]:
df_intents.head()

,phrases,class,phrases_sans_stop_words,stemmed_phrases
0,activer un renvoi d'appel,1,activer renvoi d'appel,activ renvoi d'appel
1,demande à djingo pro de renvoyer les appels,1,demande djingo pro renvoyer appels,demand djingo pro renvoi appel
2,de renvoyer les appels,1,renvoyer appels,renvoi appel
3,aide moi à renvoyer mes appels,1,aide renvoyer appels,aid renvoi appel
4,je veux renvoyer mes appels,1,veux renvoyer appels,veux renvoi appel


## TFIDF


le TFIDF est une approche bag-of-words (bow) permettant de représenter les mots d’un document à l’aide d’une matrice de nombres. Le terme bow signifie que l’ordre des mots dans la phrase n’est pas pris en compte, contrairement à des approches plus poussées de Deep Learning (word embeddings : word2vec, GLoVE). Néanmoins, TFIDF est une méthode efficace utilisée dans de nombreux outils de querying puisqu’elle donne de l’importance aux mots qui apparaissent de temps en temps mais pas trop, tout en en limitant l’importance des mots qui apparaissent souvent.

TFIDF: Cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document, relativement à une collection ou un corpus.Le poids augmente proportionnellement au nombre d'occurrences du mot dans le document. 

Term Frequency : Frequence du terme dans le document mot/tous les mots

Inverse document Frequency: 
TFIDF = Term Frequency * Inverse document frequency : log(doc/nbre de doc dans lequel apparait le terme)





In [18]:
vect = TfidfVectorizer()
mat_tfidf = vect.fit_transform(df_intents.stemmed_phrases)
#print(mat_tfidf)
#remmettre la matrice au bon endroit
feature_names = vect.get_feature_names()#to get the names of the tokens 
dense = mat_tfidf.todense() #convert sparse matrix to numpy array
denselist= dense.tolist()#convert array to list
df_tfidf= pd.DataFrame(denselist, columns = feature_names)#convert to dataframe
df_tfidf.head()

,activ,aid,annul,appel,bureau,demand,djingo,déjà,là,pro,rentr,renvoi,renvois,retour,revenu,souhait,ver,veux
0,0.887136,0.000000,0.0,0.308156,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.343554,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.192072,0.0,0.552948,0.552948,0.0,0.0,0.552948,0.0,0.214136,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,0.0,0.667716,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.744416,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.887136,0.0,0.308156,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.343554,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.413713,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.461236,0.0,0.0,0.0,0.0,0.0,0.784922


## Support vector Machine

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
#create separate object for target variable 

#split test train set
X_train, X_test, y_train,y_test = train_test_split(df_tfidf, df_intents['class'], test_size = 0.3)
print(len(X_train), len(X_test), len(y_train), len(y_test))

#
clf = LinearSVC()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
#confusion matrice
cm = confusion_matrix(y_test,y_pred)
print(cm)
#accuracy 
acc = accuracy_score(y_test,y_pred)
print(acc)


19 9 19 9
[[2 0 0]
 [0 5 0]
 [0 0 2]]
1.0


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         2

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [32]:
nouvelles_phrases = ['je veux renvoyer mes appels']
truc = steammer_phrase(nouvelle_phrase)
print(truc)

je veux renvoi me appel


In [27]:
#tfidf ne marche pas avec une phrase 
tfidf_phrase = vect.fit_transform(truc) 

ValueError: Iterable over raw text documents expected, string object received.

In [49]:
#essayons avec un doc 
nouvelles_phrases = ['je veux renvoyer mes appels','je sors du bureau','annule le renvoi d appel']
tfidf_phrases = vect.fit_transform(nouvelles_phrases)
print(tfidf_phrases)
#print(X_test)
feature_names2 = vect.get_feature_names()#to get the names of the tokens 
dense2 = tfidf_phrases.todense() #convert sparse matrix to numpy array
denselist2= dense.tolist()#convert array to list
print(feature_names2)
print(denselist2)
df_tfidf_phrase= pd.DataFrame(denselist2, columns = feature_names2)#convert to dataframe
df_tfidf_phrase.head()
#matrice_nouvelles_phrases = 

  (0, 0)	0.47107781233161794
  (0, 4)	0.47107781233161794
  (0, 5)	0.47107781233161794
  (0, 7)	0.47107781233161794
  (0, 3)	0.33517574332792605
  (1, 1)	0.534046329052269
  (1, 2)	0.534046329052269
  (1, 6)	0.534046329052269
  (1, 3)	0.37997836159100784
['appels', 'bureau', 'du', 'je', 'mes', 'renvoyer', 'sors', 'veux']
[[0.8871362336648856, 0.0, 0.0, 0.3081559081505087, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3435538373978916, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.19207216097005836, 0.0, 0.5529479363142041, 0.5529479363142041, 0.0, 0.0, 0.5529479363142041, 0.0, 0.21413552754711368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.667715722594439, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7444163578268455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.8871362336648856, 0.0, 0.3081559081505087, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3435538373978916, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.4137131759504936, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46123648912952564, 0.0, 0.0, 0.0, 0.0, 

ValueError: 8 columns passed, passed data had 18 columns

In [38]:
predicted = clf.predict(tfidf_phrases)
    

ValueError: X has 8 features per sample; expecting 18

A mon avis le model overfit car dataset trop petit essayons des algo encore plus simples exemple regression logistique
on peut aussi essayer d'entrainer sur très peu de données et tester sur plus de données 
essayer différentes méthodes de régularization
voir aussi si ça marche avec une simple liste
